In [1]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print('device = ', device)

device =  cuda


In [2]:
pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 14.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [3]:
!git clone https://github.com/gregdurrett/fp-dataset-artifacts.git

Cloning into 'fp-dataset-artifacts'...
remote: Enumerating objects: 36, done.
remote: Total 36 (delta 0), reused 0 (delta 0), pack-reused 36
Receiving objects: 100% (36/36), 14.09 KiB | 14.09 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [4]:
pip install -r ./fp-dataset-artifacts/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00


In [5]:
pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 27.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


In [6]:
cd fp-dataset-artifacts/

/content/fp-dataset-artifacts


In [7]:
from datasets import load_dataset
dataset_squad = load_dataset("squad")

# Save the 'train' split to JSON
dataset_squad["train"].to_json('squad_train.json')

# Save the 'train' split to JSON
dataset_squad["validation"].to_json('squad_test.json')

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/88 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

11139706

In [8]:
dataset_squad.shape

{'train': (87599, 5), 'validation': (10570, 5)}

In [ ]:
from datasets import load_dataset
dataset_adversarial_QA = load_dataset("adversarial_qa", "adversarialQA")

# Save the 'train' split to JSON
dataset_adversarial_QA["train"].to_json('dataset_adversarial_QA_train.json')
dataset_adversarial_QA["validation"].to_json('dataset_adversarial_QA_validation.json')
dataset_adversarial_QA["test"].to_json('dataset_adversarial_QA_test.json')

dataset_adversarial_QA.shape

Creating json from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

{'train': (30000, 6), 'validation': (3000, 6), 'test': (3000, 6)}

In [ ]:
import json

def remove_metadata_from_json(file_path):
    """
    Removes the 'metadata' dictionary from each JSON object in a file where each line is a separate JSON object.
    Saves the modified content to a new file.

    :param file_path: Path to the JSON file.
    """
    modified_data = []

    # Read the file line by line
    with open(file_path, 'r') as file:
        for line in file:
            entry = json.loads(line)
            if 'metadata' in entry:
                del entry['metadata']
            modified_data.append(entry)

    # Save the modified data to a new file
    modified_file_path = file_path.replace('.json', '_modified.json')
    with open(modified_file_path, 'w') as file:
        for entry in modified_data:
            json.dump(entry, file)
            file.write('\n')  # Write each JSON object on a new line

    return modified_file_path

In [ ]:
file_path = 'dataset_adversarial_QA_train.json'
modified_file_path = remove_metadata_from_json(file_path)

In [9]:
from datasets import load_dataset

# Load the dataset
dataset_AdQuest_parent = load_dataset("mbartolo/synQA")

# Define a filter function
def starts_with_specific_words(example):
    question_start_words = ["when", "where", "which", "how many"]
    question_lower = example['question'].lower()
    return any(question_lower.startswith(word) for word in question_start_words)

# Filter the dataset
AdQuest = dataset_AdQuest_parent['train'].filter(starts_with_specific_words)

# Save the filtered 'train' split to JSON
AdQuest.to_json('AdQuest_train.json')

# Check the shape of the filtered dataset
AdQuest.shape


Generating train split:   0%|          | 0/314811 [00:00<?, ? examples/s]

Filter:   0%|          | 0/314811 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/68 [00:00<?, ?ba/s]

(67945, 5)

In [ ]:
from datasets import load_dataset
import pandas as pd
import plotly.express as px
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
import plotly.io as pio

def process_questions(questions):
    processed_data = []
    for question in questions:
        words = word_tokenize(question)[:3]
        for i in range(2, len(words)):
            grandparent = words[i-2]
            parent = words[i-1]
            word = words[i]
            processed_data.append({'grandparent': grandparent, 'parent': parent, 'word': word})
    return processed_data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
dataset_AdQuest = load_dataset("mbartolo/synQA")

# Combine train and validation questions
AdQuest_train_questions = dataset_AdQuest['train']['question']

processed_data_AdQuest_train = process_questions(AdQuest_train_questions)

# Creating a DataFrame for the sunburst plot
AdQuest_Train_DF = pd.DataFrame(processed_data_AdQuest_train)

# Convert 'grandparent', 'parent', and 'word' columns to lowercase to avoid case-sensitive distinctions
AdQuest_Train_DF['grandparent'] = AdQuest_Train_DF['grandparent'].str.lower()
AdQuest_Train_DF['parent'] = AdQuest_Train_DF['parent'].str.lower()
AdQuest_Train_DF['word'] = AdQuest_Train_DF['word'].str.lower()

# Aggregate and count the frequency of each triplet
AdQuest_Train_agg_DF = AdQuest_Train_DF.groupby(['grandparent', 'parent', 'word']).size().reset_index(name='frequency')

In [ ]:
import plotly.io as pio

# Sort by frequency and take the top 100 rows
AdQuest_Train_agg_DF_head = AdQuest_Train_agg_DF.sort_values('frequency', ascending=False).head(10000)

# Generate sunburst plot using the aggregated frequency of the top 100 rows
fig = px.sunburst(
    AdQuest_Train_agg_DF_head,
    path=['grandparent', 'parent', 'word'],
    values='frequency',
    title="Sunburst Plot of Word Triplets in Questions"
)
fig.update_layout(width=800, height=800,
                  paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')

# Save the figure as a PNG file
file_path = "AdQuest_Train_Word_Distribution.png"  # You can specify a different path or filename
pio.write_image(fig, file_path)

fig.show()

In [ ]:
processed_data_AdQuest_train = process_questions(AdQuest['question'])

# Creating a DataFrame for the sunburst plot
AdQuest_Train_DF = pd.DataFrame(processed_data_AdQuest_train)

# Convert 'grandparent', 'parent', and 'word' columns to lowercase to avoid case-sensitive distinctions
AdQuest_Train_DF['grandparent'] = AdQuest_Train_DF['grandparent'].str.lower()
AdQuest_Train_DF['parent'] = AdQuest_Train_DF['parent'].str.lower()
AdQuest_Train_DF['word'] = AdQuest_Train_DF['word'].str.lower()

# Aggregate and count the frequency of each triplet
AdQuest_Train_agg_DF = AdQuest_Train_DF.groupby(['grandparent', 'parent', 'word']).size().reset_index(name='frequency')

In [ ]:
import plotly.express as px
import plotly.io as pio

# Assuming AdQuest_Train_agg_DF is your DataFrame
# Sort by frequency and take the top 100 rows
AdQuest_Train_agg_DF_head = AdQuest_Train_agg_DF.sort_values('frequency', ascending=False).head(10000)

# Generate sunburst plot using the aggregated frequency of the top 100 rows
fig = px.sunburst(
    AdQuest_Train_agg_DF_head,
    path=['grandparent', 'parent', 'word'],
    values='frequency',
    title="AdQuest question distribution"
)

# Update layout and make labels bold
fig.update_traces(insidetextfont={'color':'black', 'size':18, 'family':'Arial, bold'})

fig.update_layout(width=800, height=800,
                  paper_bgcolor='rgba(0,0,0,0)',plot_bgcolor='rgba(0,0,0,0)')

# Save the figure as a PNG file
file_path = "AdQuest_Train_Word_Distribution.png"  # You can specify a different path or filename
pio.write_image(fig, file_path)

fig.show()

In [ ]:
!cat AdQuest_train.json squad_train.json > dataset_train.json

In [ ]:
!python3 run.py --do_train --num_train_epochs 3 --task qa --dataset dataset_train.json --output_dir ./trained_AdQuest_where_when_how_many_model/

2023-12-05 02:22:28.096382: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 02:22:28.096439: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 02:22:28.096469: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 02:22:29.231107: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Extracting data files: 100% 1/1 [00:00<00:00, 1436.41it/s]
Generating train split: 155544 examples [00:00, 353855.90 examples/s]
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-

In the next step, three datasets are combined to create one superset of dataset for training. It includes the squad_train.json, dataset_synQA_train.json, and dataset_adversarial_QA_train_modified.json.

In [ ]:
file_path = 'dataset_adversarial_QA_validation.json'
modified_file_path = remove_metadata_from_json(file_path)

In [ ]:
!python3 run.py --do_eval --task qa --dataset squad_test.json --model ./trained_AdQuest_where_when_how_many_model/ --output_dir ./eval_AdQuest_where_when_how_many_model_on_SQuAD/

2023-12-05 03:35:57.988760: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 03:35:57.988812: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 03:35:57.988839: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 03:35:59.124881: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Extracting data files: 100% 1/1 [00:00<00:00, 221.66it/s]
Generating train split: 10570 examples [00:00, 112023.61 examples/s]
Preprocessing data... (this takes a little bit, should only happen once per dataset)
Map (num_proc=2): 100% 105

In [ ]:
# Load the dataset
dataset_AdverseQA_Robin = load_dataset("squad_adversarial", "AddSent")
dataset_AdverseQA_Robin["validation"].to_json('dataset_AdverseQA_Robin.json')
dataset_AdverseQA_Robin.shape

Creating json from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

{'validation': (3560, 5)}

In [ ]:
# Extract contexts from the validation split
contexts = dataset_AdverseQA_Robin["validation"]["context"]

# Count unique contexts
unique_contexts = set(contexts)
num_unique_contexts = len(unique_contexts)

print(f"Number of unique contexts in the dataset: {num_unique_contexts}")
print(len(contexts))

Number of unique contexts in the dataset: 3356
3560


In [ ]:
!python3 run.py --do_eval --task qa --dataset dataset_AdverseQA_Robin.json --model ./trained_AdQuest_where_when_how_many_model/ --output_dir ./eval_AdverseQA_Robin/

2023-12-05 03:50:47.059056: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 03:50:47.059106: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 03:50:47.059138: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 03:50:48.189084: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Extracting data files: 100% 1/1 [00:00<00:00, 1472.20it/s]
Generating train split: 3560 examples [00:00, 200522.70 examples/s]
Preprocessing data... (this takes a little bit, should only happen once per dataset)
Map (num_proc=2): 100% 356

In [ ]:
!cat filtered_dataset_synQA_train.json squad_train.json dataset_adversarial_QA_train_modified.json > dataset_AdQuest_SQuAD_AQA_train.json

In [ ]:
!python3 run.py --do_train --num_train_epochs 3 --task qa --dataset dataset_AdQuest_SQuAD_AQA_train.json --output_dir ./AdQuest_SQuAD_AQA/

2023-12-05 04:17:42.279821: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 04:17:42.279883: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 04:17:42.279915: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 04:17:43.434019: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Extracting data files: 100% 1/1 [00:00<00:00, 1526.87it/s]
Generating train split: 185544 examples [00:00, 360869.94 examples/s]
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at google/electra-

In [ ]:
!python3 run.py --do_eval --task qa --dataset squad_test.json --model ./AdQuest_SQuAD_AQA/ --output_dir ./eval_ADQuest_SQuAD_AdverseQA/

2023-12-05 05:35:50.137660: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 05:35:50.137707: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 05:35:50.137734: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 05:35:51.514945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Preprocessing data... (this takes a little bit, should only happen once per dataset)
Map (num_proc=2): 100% 10570/10570 [00:05<00:00, 1992.74 examples/s]
/content/fp-dataset-artifacts/run.py:137: FutureWarning: load_metric is deprecated a

In [ ]:
!python3 run.py --do_eval --task qa --dataset dataset_AdverseQA_Robin.json --model ./AdQuest_SQuAD_AQA/ --output_dir ./eval_AdQuest_SQuAD_AdverseQA_Robin/

2023-12-05 05:39:25.116230: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-05 05:39:25.116279: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-05 05:39:25.116305: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-05 05:39:26.242256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Preprocessing data... (this takes a little bit, should only happen once per dataset)
Map (num_proc=2): 100% 3560/3560 [00:01<00:00, 1920.02 examples/s]
/content/fp-dataset-artifacts/run.py:137: FutureWarning: load_metric is deprecated and

In [11]:
!zip -r /content/fp-dataset-artifacts.zip /content/fp-dataset-artifacts/

  adding: content/fp-dataset-artifacts/ (stored 0%)
  adding: content/fp-dataset-artifacts/.git/ (stored 0%)
  adding: content/fp-dataset-artifacts/.git/branches/ (stored 0%)
  adding: content/fp-dataset-artifacts/.git/index (deflated 36%)
  adding: content/fp-dataset-artifacts/.git/hooks/ (stored 0%)
  adding: content/fp-dataset-artifacts/.git/hooks/pre-commit.sample (deflated 45%)
  adding: content/fp-dataset-artifacts/.git/hooks/post-update.sample (deflated 27%)
  adding: content/fp-dataset-artifacts/.git/hooks/pre-merge-commit.sample (deflated 39%)
  adding: content/fp-dataset-artifacts/.git/hooks/commit-msg.sample (deflated 44%)
  adding: content/fp-dataset-artifacts/.git/hooks/fsmonitor-watchman.sample (deflated 62%)
  adding: content/fp-dataset-artifacts/.git/hooks/applypatch-msg.sample (deflated 42%)
  adding: content/fp-dataset-artifacts/.git/hooks/push-to-checkout.sample (deflated 55%)
  adding: content/fp-dataset-artifacts/.git/hooks/update.sample (deflated 68%)
  adding: co